<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<br><b>Comparing Object and Truth Tables</b> <br>
Contact author: Jeff Carlin <br>
Last verified to run: 2022-09-29 <br>
LSST Science Piplines version: Weekly 2022_40 <br>
Container size: medium <br>
Targeted learning level: beginner <br>

**Description:** An introduction to using the truth data for the Dark Energy Science Collaboration's DC2 data set, which formed the basis for the DP0 data products.

**Skills:** Use the TAP service with table joins to retrieve truth data matched to the Object catalog.

**LSST Data Products:** TAP dp02_dc2_catalogs.Object, .MatchesTruth, and .TruthSummary tables. 

**Packages:** lsst.rsp.get_tap_service, lsst.rsp.retrieve_query

**Credit:** Originally developed by Jeff Carlin and the Rubin Community Engagement Team in the context of the Rubin DP0, with some help from Melissa Graham in the update from DP0.1 to DP0.2.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0-2.lsst.io">dp0-2.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1.0. Introduction

This tutorial demonstrates how to use the TAP service to query and retrieve data from the two truth tables and the Object table for DP0.2.
Joining these three tables enables users to compare the recovered (measured) properties (e.g., fluxes, positions, magnitudes, etc. from the Object table, which is comprised of SNR>5 detections in the deepCoadds) to the simulated values that were assigned to each object when creating the DC2 simulations.

More information about the DC2 simulations that make up DP0 can be found in [the DC2 Data Release Note](https://ui.adsabs.harvard.edu/abs/2021arXiv210104855L/abstract).

### 1.1. Package imports

The [`matplotlib`](https://matplotlib.org/), [`numpy`](http://www.numpy.org/), [`pandas`](https://pandas.pydata.org/docs/), and [`astropy`](http://www.astropy.org/) libraries are widely used Python libraries for plotting, scientific computing, and astronomical data analysis. We will use these packages below, including the `matplotlib.pyplot` plotting sublibrary.

We also use the `warnings` package to minimize standard output within the notebook, and the `lsst.rsp` package to access the TAP service and query the DP0 catalogs.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas
from astropy.units import UnitsWarning
import warnings
from lsst.rsp import get_tap_service, retrieve_query

### 1.2. Define functions and parameters

Ignore units warnings in this notebook, they are not cricital.

Set the pandas parameter for the maximum number of rows to display to 200.

In [ ]:
warnings.simplefilter("ignore", category=UnitsWarning)
pandas.set_option('display.max_rows', 200)

Set `matplotlib` to show plots inline, within the notebook.

In [ ]:
%matplotlib inline

Set up colors and plot symbols corresponding to the _ugrizy_ bands. These colors are the same as those used for _ugrizy_ bands in Dark Energy Survey (DES) publications, and are defined in <a href="https://github.com/DarkEnergySurvey/descolors">this github repository</a>.

In [ ]:
plot_filter_labels = ['u', 'g', 'r', 'i', 'z', 'y']
plot_filter_colors = {'u': '#56b4e9', 'g': '#008060', 'r': '#ff4000',
                      'i': '#850000', 'z': '#6600cc', 'y': '#000000'}
plot_filter_symbols = {'u': 'o', 'g': '^', 'r': 'v', 'i': 's', 'z': '*', 'y': 'p'}

To access tables, we will use the TAP service in a similar manner to what we showed in the [Intro to DP0 notebook](https://github.com/rubin-dp0/tutorial-notebooks/blob/main/01_Intro_to_DP0_Notebooks.ipynb), and explored further in the [TAP tutorial notebook](https://github.com/rubin-dp0/tutorial-notebooks/blob/main/02_Intermediate_TAP_Query.ipynb). See those notebooks for more details.

In [ ]:
service = get_tap_service()

## 2.0. Discover truth data

The <a href="dp0-2.lsst.io">DP0.2 Documentation</a> contains a <a href="https://dp0-2.lsst.io/data-products-dp0-2/index.html#catalogs">list of all DP0.2 catalogs</a>, and also a link to the <a href="https://dm.lsst.org/sdm_schemas/browser/dp02.html">DP0.2 Schema Browser</a> where users can read about the available tables and their contents.

Alternatively, the Portal Aspect of the Rubin Science Platform can be used to browse catalog data.

Below, we show how to browse catalog data from a Notebook using the TAP service.

### 2.1. Print the names of all available tables

In [ ]:
results = service.search("SELECT description, table_name FROM TAP_SCHEMA.tables")
results_tab = results.to_table()

for tablename in results_tab['table_name']:
    print(tablename)

In [ ]:
del results, results_tab

### 2.2. Print the table schema for MatchesTruth

Use the `.to_pandas()` method, and not just `.to_table()` (astropy table), so that all rows of the second cell display.

In [ ]:
results = service.search("SELECT column_name, datatype, description,\
                          unit from TAP_SCHEMA.columns\
                          WHERE table_name = 'dp02_dc2_catalogs.MatchesTruth'")

In [ ]:
results.to_table().to_pandas()

The above is fine if the full description is not needed, but there are some important details that are being hidden by the line truncation above.

Try this instead.

In [ ]:
for c, columnname in enumerate(results['column_name']):
    print('%-25s %-200s' % (columnname, results['description'][c]))

In [ ]:
del results

### 2.2. Print the table schema for TruthSummary

In [ ]:
results = service.search("SELECT column_name, datatype, description,\
                          unit from TAP_SCHEMA.columns\
                          WHERE table_name = 'dp02_dc2_catalogs.TruthSummary'")

In [ ]:
results.to_table().to_pandas()

In [ ]:
for c, columnname in enumerate(results['column_name']):
    print('%-25s %-200s' % (columnname, results['description'][c]))

In [ ]:
del results

## 3.0. Retrieve truth data

### 3.1. Join MatchesTruth and TruthSummary

As described in the column description, the column `id_truth_type` should be used to join the `MatchesTruth` and `TruthSummary` tables.

In [ ]:
%%time
query = "SELECT mt.id_truth_type, mt.match_objectId, ts.ra, ts.dec, ts.truth_type "\
        "FROM dp02_dc2_catalogs.MatchesTruth AS mt "\
        "JOIN dp02_dc2_catalogs.TruthSummary AS ts ON mt.id_truth_type = ts.id_truth_type "\
        "WHERE CONTAINS(POINT('ICRS', ts.ra, ts.dec), CIRCLE('ICRS', 62.0, -37.0, 0.10)) = 1 "
print(query)
print(' ')
results = service.search(query)

In [ ]:
results.to_table()

Notice that not all objects from the truth table have matches in the dp02_dc2_catalogs.Object table of detections in the deep coadded images (i.e., their "match_objectId" is blank, meaning there was no match). Print the fraction of retrieved truth objects that are matched to the dp02_dc2_catalogs.Object table.

In [ ]:
tx = np.where(results['match_objectId'] > 1)[0]
print('Number: ', len(tx))
print('Fraction: ', np.round(len(tx)/len(results),2))

In [ ]:
del results

### 3.2. Triple-join MatchesTruth, TruthSummary, and Objects

The `MatchesTruth` table provides identifying information to pick out objects that have matches to truth objects. In order to compare _measured_ quantities (e.g., fluxes) from the `Object` table to the simulated truth values, we also need data from the `TruthSummary` table. This requires joining all three tables.

Note that the table length is now equal to the number of matched objects printed above (14850).

In [ ]:
%%time
query = "SELECT mt.id_truth_type, mt.match_objectId, ts.ra, ts.dec, ts.truth_type, "\
        "obj.coord_ra, obj.coord_dec "\
        "FROM dp02_dc2_catalogs.MatchesTruth AS mt "\
        "JOIN dp02_dc2_catalogs.TruthSummary AS ts ON mt.id_truth_type = ts.id_truth_type "\
        "JOIN dp02_dc2_catalogs.Object AS obj ON mt.match_objectId = obj.objectId "\
        "WHERE CONTAINS(POINT('ICRS', ts.ra, ts.dec), CIRCLE('ICRS', 62.0, -37.0, 0.10)) = 1 "
results = service.search(query)
results.to_table()

In [ ]:
del results

### 3.3. Retrieve additional data for true galaxies that are matched to detected objects

With the query below we retrieve much more data for true galaxies, such as their true and measured fluxes and extendedness. Since we are only retrieving measurement data from the Object table for true galaxies, we retrieve the `cModelFlux` instead of the `psfFlux`, the latter being appropriate for point-sources (e.g., stars).

> **Notice:** Above, the column names in the retrieved results have no provenance: once the data is in the results table, it is unclear from which table it originated (i.e., whether it is a true coordinate or a measured coordinate). Below, we use the `AS` statement to rename columns to start with their origin table, in order to keep track of what is from a truth table (mt_ and ts_) and what is from the object table (obj_).

> **Notice:** Below, we use `truth_type = 1` to only retrieve truth and measurement data for "true galaxies."

The following query will retrieve 14501 results, 349 fewer than the query above, due to the specification of `truth_type = 1`.

In [ ]:
query = "SELECT mt.id_truth_type AS mt_id_truth_type, "\
        "mt.match_objectId AS mt_match_objectId, "\
        "ts.ra AS ts_ra, "\
        "ts.dec AS ts_dec, "\
        "ts.truth_type AS ts_truth_type, "\
        "ts.mag_r AS ts_mag_r, "\
        "ts.is_pointsource AS ts_is_pointsource, "\
        "ts.redshift AS ts_redshift, "\
        "ts.flux_u AS ts_flux_u, "\
        "ts.flux_g AS ts_flux_g, "\
        "ts.flux_r AS ts_flux_r, "\
        "ts.flux_i AS ts_flux_i, "\
        "ts.flux_z AS ts_flux_z, "\
        "ts.flux_y AS ts_flux_y, "\
        "obj.coord_ra AS obj_coord_ra, "\
        "obj.coord_dec AS obj_coord_dec, "\
        "obj.refExtendedness AS obj_refExtendedness, "\
        "scisql_nanojanskyToAbMag(obj.r_cModelFlux) AS obj_cModelMag_r, "\
        "obj.u_cModelFlux AS obj_u_cModelFlux, "\
        "obj.g_cModelFlux AS obj_g_cModelFlux, "\
        "obj.r_cModelFlux AS obj_r_cModelFlux, "\
        "obj.i_cModelFlux AS obj_i_cModelFlux, "\
        "obj.z_cModelFlux AS obj_z_cModelFlux, "\
        "obj.y_cModelFlux AS obj_y_cModelFlux "\
        "FROM dp02_dc2_catalogs.MatchesTruth AS mt "\
        "JOIN dp02_dc2_catalogs.TruthSummary AS ts ON mt.id_truth_type = ts.id_truth_type "\
        "JOIN dp02_dc2_catalogs.Object AS obj ON mt.match_objectId = obj.objectId "\
        "WHERE CONTAINS(POINT('ICRS', ts.ra, ts.dec), CIRCLE('ICRS', 62.0, -37.0, 0.10)) = 1 "\
        "AND ts.truth_type = 1 "\
        "AND obj.detect_isPrimary = 1"
print(query)

This query might take a couple of minutes to execute.

In [ ]:
%%time
results = service.search(query)

In [ ]:
results.to_table()

Notice that there is no `del results` statement here.

Keep these results and use them below, in Section 4, to explore the retrieved data for true galaxies.

## 4.0. Compare true and measured properties for true galaxies

### 4.1. Plot coordinate offsets for true galaxies

Below, plot the difference between the true and measured declination versus the difference between the true and measured right ascension. 

In [ ]:
fig = plt.figure(figsize=(4, 4))
plt.plot(3600*(results['ts_ra']-results['obj_coord_ra']), \
         3600*(results['ts_dec']-results['obj_coord_dec']), \
         'o', ms=2, alpha=0.2, mew=0)
plt.xlabel('Right Ascension (true-measured; ["])', fontsize=12)
plt.ylabel('Declination (true-measured; ["])', fontsize=12)
plt.show()

We see that the scatter is less than about 0.5 arcseconds. For the (DC2-simulated) LSST Science Camera's platescale of 0.2 arcsec per pixel, that's a measurement accuracy of 2.5 pixels. Note also that most galaxies' positions are measured to sub-pixel accuracy.

### 4.2. How many true galaxies are measured as point sources?

The number of true galaxies that are measured as point sources, or in other words, have a measured `refExtendedness` equal to zero (see the [schema for the DP0.2 Object table](https://dm.lsst.org/sdm_schemas/browser/dp02.html#Object) for more info about the `refExtendedness` column).

In this example, the following cell will report that 19% of true galaxies appear as point sources.

In [ ]:
x = np.where(results['obj_refExtendedness'] == 0)[0]
print('Number: ', len(x))
print('Fraction: ', np.round(len(x)/len(results['ts_is_pointsource']),2))
del x

The fact that 19% of the true galaxies retrieved from the catalog appear point-like does not necessarily indicate an error in the measurement pipelines. For example, very small or very distant galaxies can appear point-like, even if they were simulated as extended objects. It is left as an exercise for the learner to explore what types of galaxies are measured to be point-like.

### 4.3. Compare true and measured r-band magnitudes for true galaxies

In [ ]:
fig = plt.figure(figsize=(4, 4))
plt.plot([18,32], [18,32], ls='solid', color='black', alpha=0.5)
x = np.where(results['obj_refExtendedness'] == 1)[0]
plt.plot(results['ts_mag_r'][x], results['obj_cModelMag_r'][x], \
         'o', ms=4, alpha=0.2, mew=0, color=plot_filter_colors['r'],\
         label='measured as extended')
del x
x = np.where(results['obj_refExtendedness'] == 0)[0]
plt.plot(results['ts_mag_r'][x], results['obj_cModelMag_r'][x], \
         'o', ms=2, alpha=0.5, mew=0, color='black',\
         label='measured as point-like')
del x
plt.xlabel('true r-band magnitude', fontsize=12)
plt.ylabel('measured cModel r-band magnitude', fontsize=12)
plt.legend(loc='lower right')
plt.xlim([18,30])
plt.ylim([18,30])
plt.show()

### 4.4. Compare true and measured fluxes in all filters for true galaxies

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(10, 7))
i=0
j=0
for f,filt in enumerate(plot_filter_labels):
    ax[i,j].plot([0.1,1e6], [0.1,1e6], ls='solid', color='black', alpha=0.5)
    ax[i,j].plot(results['ts_flux_'+filt], results['obj_'+filt+'_cModelFlux'], \
                 plot_filter_symbols[filt], color=plot_filter_colors[filt], \
                 alpha=0.1, mew=0, label=filt)
    ax[i,j].loglog()
    ax[i,j].text(0.1, 0.9, filt, horizontalalignment='center', verticalalignment='center',
                 transform = ax[i,j].transAxes, color=plot_filter_colors[filt], fontsize=14)
    ax[i,j].set_xlim([0.1,1e6])
    ax[i,j].set_ylim([0.1,1e6])
    j += 1
    if j == 3:
        i += 1
        j = 0
ax[0,0].set_ylabel('measured cModelFlux', fontsize=12)
ax[1,0].set_ylabel('measured cModelFlux', fontsize=12)
ax[1,0].set_xlabel('true flux', fontsize=12)
ax[1,1].set_xlabel('true flux', fontsize=12)
ax[1,2].set_xlabel('true flux', fontsize=12)
plt.tight_layout()
plt.show()

### 4.5. Compare color-magnitude diagrams (CMDs) for true and measured properties of true galaxies

The following cells plot the true CMD at left in black, and the measured CMD at right in grey.

The first pair of plots uses the r-band for magnitude, and the g-r color. The second pair of plots uses the r-band for magnitude, and i-z for color.

In the first set of plots, the effects of measurement uncertainties are correlated between the _x_ and _y_ axes because the r-band data is included in both axes. In the second set of plots, the i-band and the z-band are instead used for color. Notice how the effect of measurement uncertainties changes.

Recall that these plots do not contain data for stars, as only true galaxies were retrieved from the truth tables.

> **Warning:** Pink "RuntimeWarning" errors will appear due to a few of the measured fluxes in the denominator being zero. It is OK to ignore these warnings for the context of this tutorial, which focuses on retrieving truth data, but for scientific analyses users should follow up and understand such warnings (e.g., use flags to reject poor flux measurements from their samples).

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].plot(-2.5*np.log10(results['ts_flux_g']/results['ts_flux_r']), results['ts_mag_r'], \
           'o', ms=2, alpha=0.2, mew=0, color='black')

ax[1].plot(-2.5*np.log10(results['obj_g_cModelFlux']/results['obj_r_cModelFlux']), results['obj_cModelMag_r'], \
           'o', ms=2, alpha=0.2, mew=0, color='grey')
ax[0].set_xlabel('true color (g-r)', fontsize=12)
ax[0].set_ylabel('true magnitude (r-band)', fontsize=12)
ax[0].set_xlim([-2, 4])
ax[0].set_ylim([30, 18])
ax[1].set_xlabel('measured color (g-r)', fontsize=12)
ax[1].set_ylabel('measured magnitude (r-band)', fontsize=12)
ax[1].set_xlim([-2, 4])
ax[1].set_ylim([30, 18])
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].plot(-2.5*np.log10(results['ts_flux_i']/results['ts_flux_z']), results['ts_mag_r'], \
           'o', ms=2, alpha=0.2, mew=0, color='black')

ax[1].plot(-2.5*np.log10(results['obj_i_cModelFlux']/results['obj_z_cModelFlux']), results['obj_cModelMag_r'], \
           'o', ms=2, alpha=0.2, mew=0, color='grey')
ax[0].set_xlabel('true color (i-z)', fontsize=12)
ax[0].set_ylabel('true magnitude (r-band)', fontsize=12)
ax[0].set_xlim([-2, 4])
ax[0].set_ylim([30, 18])
ax[1].set_xlabel('measured color (i-z)', fontsize=12)
ax[1].set_ylabel('measured magnitude (r-band)', fontsize=12)
ax[1].set_xlim([-2, 4])
ax[1].set_ylim([30, 18])
plt.tight_layout()
plt.show()

## 5.0 Exercises for the learner

1. Repeat the query in Section 3.3, but instead of only retrieving true galaxies (`ts.truth_type = 1`), include stars, which have a `truth_type` of 2. Since stars are point sources, instead of only retrieving `cModelFlux` measurements, also retrieve `psfFlux` from the Object catalog, because PSF-fit fluxes are more appropriate for point sources.

2. As mentioned in Section 4.2, it is left as an exercise for the learner to explore what types of galaxies are measured to be point-like.

3. Explore the truth data for Type Ia supernovae (`truth_type = 3`).